During the last couple of months I've started to use the Jupyter notebook (formerly iPython notebook) for more and more stuff. Inspired by [Python Perambulations](https://jakevdp.github.io/blog/2013/05/07/migrating-from-octopress-to-pelican/) by Jake VanderPlas, I decided to also try to use it for blogging. In the following I'll try to document how I've setup things. 

## Tools involved

* [Conda](http://conda.pydata.org/docs/): Package manager which we will used to install all our tools. 
* [Pelican](http://getpelican.com): Pelican is a static site generator, written in Python. 
* [Jupyter](http://jupyter.org): What we will use to write our notebooks in.

## Installing Conda

Conda is a package manager which allows us to install various versions of different libraries and applications in isolated environments (much like pip and virtualenv known from the Python community). I've done a seperate post on how I've setup Conda here, so I will not go into it more here. 

Conda has the concept of *environments* which allows us to install and use several different versions of libraries and applications side-by-side. E.g. in one environment we may have Python 2.7 installend and in another Python 3.4. To create an Conda environment we need to find Conda packages for all the libraries and applications we need. Conda allows us to read and create environments based on an `environment.yml` file. You can pin down the exact versions of all the dependencies in the environment if you like (to create a fully reproducible setup), but in this case I didn't see the need for that.  

```
name: blog
channels:
  - https://conda.anaconda.org/davethecipo
dependencies:
  - pelican
  - jupyter
  - markdown
```

You can see the up-to-date version of the `environment.yml` in the blog repository [here](https://github.com/mortenvp/mortenvp-pelican). 

The above file contains three sections:

1. `name`: This is going to be the name of the environment we create.
2. `channels`: Allows us to use *user provided* packages. In this specific case I want to use a version of pelican provided by the user *davethecipo*. You can search
3. `dependencies`: These are the names of the packages we want to install.

To create the blog environment run (in the same folder as our `environment.yml`):

```
conda env create
```

This will create the `blog` environment, which you can active by running:

```
conda activate blog
```

Similarly you can deactivate it by running:

```
conda deactivate
```

You can list your currently available environments:

```
conda env list
```

And if you want to delete an environment e.g. `NAME` you can run:

```
conda env remove -n NAME
```

## Hosting the site

Here I will again follow the approach taken by Jake VanderPlas for his [Python Perambulations](http://jakevpd.github.io) blog. 

The basic idea is to create two [GitHub](http://github.com) reporitories. The first contains the Pelican project files, notebooks, and other material used when generating the site. The second contains the generated html files etc. With this approach we can use [GitHub Pages](https://pages.github.com/) for the second repository to make the site available on the web.

The first repoitory I named [mortenvp-pelican](https://github.com/mortenvp/mortenvp-pelican) and the second [mortenvp.github.io](https://github.com/mortenvp/mortenvp.github.io) which according to the GitHub Pages documentation is the mandtory name for a personal pages repository.

## Generating the initial project

In this case I just follow the Pelican [quickstart](http://docs.getpelican.com/en/3.5.0/quickstart.html) section to setup a basic site.

```
git clone <git-url-to-mortenvp-pelican.git>
cd mortenvp-pelican
pelican-quickstart
```

The quick start will ask you a bunch of questions, these were my answers:

```
Welcome to pelican-quickstart v3.4.0.

This script will help you create a new Pelican-based website.

Please answer the following questions so this script can generate the files
needed by Pelican.

    
> Where do you want to create your new web site? [.] 
> What will be the title of this web site? blog.mortenvp.com
> Who will be the author of this web site? Morten V. Pedersen
> What will be the default language of this web site? [en] 
> Do you want to specify a URL prefix? e.g., http://example.com   (Y/n) 
> What is your URL prefix? (see above example; no trailing slash) http://blog.mortenvp.com
> Do you want to enable article pagination? (Y/n) 
> How many articles per page do you want? [10] 
> Do you want to generate a Fabfile/Makefile to automate generation and publishing? (Y/n) 
> Do you want an auto-reload & simpleHTTP script to assist with theme and site development? (Y/n) 
> Do you want to upload your website using FTP? (y/N) 
> Do you want to upload your website using SSH? (y/N) 
> Do you want to upload your website using Dropbox? (y/N) 
> Do you want to upload your website using S3? (y/N) 
> Do you want to upload your website using Rackspace Cloud Files? (y/N) 
> Do you want to upload your website using GitHub Pages? (y/N) Y
> Is this your personal page (username.github.io)? (y/N) y
Done. Your new project is available at /home/mvp/dev/mortenvp-pelican
```
## Plugin enabeling Jupyter notebooks

Support for including a Jupyter notebook is available in the [Liquid-style Tags](https://github.com/getpelican/pelican-plugins/tree/master/liquid_tags) plugin. Following the [README](https://github.com/getpelican/pelican-plugins) in the Pelican Plugins we add the pelican-plugins repository as a submodule:

```
git submodule add git@github.com:getpelican/pelican-plugins.git
```

and activate the liquid-style Tags:

```
PLUGIN_PATHS = ['pelican-plugins']
PLUGINS = ['liquid_tags.img', 'liquid_tags.video',
           'liquid_tags.youtube', 'liquid_tags.vimeo',
           'liquid_tags.include_code', 'liquid_tags.notebook']
```

You can choose the location for the notebooks by specifying:

```
NOTEBOOK_DIR = 'notebooks'
```

### Updating the plug-ins

At some point you will probably want to update the plug-ins. Since it is simply a git submodule the following should suffice: 

```
cd pelican-plugins
git pull
cd ..
git commit -m "Updating plug-ins"
```

Of course if you don't want to use the latest and greatest of the plug-ins you can always checkout a specific commit and use that. 

### Including the notebook CSS in our choosen theme

At the top of the pelicanconf.py add:

```
from __future__ import unicode_literals
import os
```

Which enables us to use (copied from [here](https://github.com/jakevdp/PythonicPerambulations/blob/master/pelicanconf.py)):

```
# The theme file should be updated so that the base header contains the line:
#
#  {% if EXTRA_HEADER %}
#    {{ EXTRA_HEADER }}
#  {% endif %}
# 
# This header file is automatically generated by the notebook plugin
if not os.path.exists('_nb_header.html'):
    import warnings
    warnings.warn("_nb_header.html not found.  "
                  "Rerun make html to finalize build.")
else:
    EXTRA_HEADER = open('_nb_header.html').read()
    
```

## Choosing a theme

I'm not a big fan of the default Pelican theme, so instead I chose to use the [Pelican-sober](https://github.com/fle/pelican-sober) theme.

Since the `EXTRA_HEADER` section (as described above) isn't present in the Pelican-sober theme's  `templates/base.html`, we need to add it. To do this I forked the repository to my user and made the changes there. 

Once this is done we can use our chosen theme by editing the `THEME` variable in `pelican.conf` (read about this [here](http://nafiulis.me/making-a-static-blog-with-pelican.html#themes)), so to summarize:

1. Select a theme and ensure that it has the `EXTRA_HEADER` section.
1. Add the theme git repository as a submodule to our site repository `git submodule add git@github.com:url-to-repo.git`
2. Add a `THEME = 'pelican-sober'` vaiable inside `pelicanconf.py`


## Adding a post

Adding a post is pretty easy, create a simple markdown file with some basic tags. As an example `constent/first_post.md`:

```
Title: My fist post
Date: 2015-02-26 15:00

# About
Here we can write the blog post using markdown
```

## Including a notebook

To include a notebook called `filename.ipynb` copy it to the `content/notebooks/` directory and add it to a post using:

```
{% notebook filename.ipynb %}
```

## Generating the site 

See Makefile